# How to get maximum comments on Hacker News site

In this project, we'll compare two different types of posts from [Hacker News](https://news.ycombinator.com/), a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either `Ask HN` or `Show HN`.

Users submit `Ask HN` posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction
First, we'll read in the data and remove the headers.

In [1]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers, '\n')
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Let's start by exploring the number of comments for each type of post.

## Extracting Ask HN and Show HN Posts
As, we are only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing data just for those type of posts. Separating the data makes it easier to analyze in the following steps.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lowercase = title.lower()
    if title_lowercase.startswith("ask hn"):
        ask_posts.append(row)
    elif title_lowercase.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print("# of Ask HN posts:", len(ask_posts))
print("# of Show HN posts:", len(show_posts))
print("# of Other posts:", len(other_posts))

# of Ask HN posts: 1744
# of Show HN posts: 1162
# of Other posts: 17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts
Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
# Calculate the average number of comments `Ask HN` posts receive
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments/ len(ask_posts)
print('Avg # of comments for "Ask HN" posts: {:.2f}'.format(avg_ask_comments))


# Calculate the average number of comments `Show HN` posts receive
total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments/ len(show_posts)
print('Avg # of comments for "Show HN" posts: {:.2f}'.format(avg_show_comments))


Avg # of comments for "Ask HN" posts: 14.04
Avg # of comments for "Show HN" posts: 10.32


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created
Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [5]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time_stamp = row[0]
    comments = row[1]
    datetime_obj = dt.datetime.strptime(time_stamp, "%m/%d/%Y %H:%M")
    hour = datetime_obj.strftime("%H") #.hour will convert it to int
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comments

counts_by_hour

{'09': 45,
 '13': 85,
 '10': 59,
 '14': 107,
 '16': 108,
 '23': 68,
 '12': 73,
 '17': 100,
 '15': 116,
 '21': 109,
 '20': 80,
 '02': 58,
 '18': 109,
 '03': 54,
 '05': 46,
 '19': 110,
 '01': 60,
 '22': 71,
 '08': 48,
 '04': 47,
 '00': 55,
 '06': 44,
 '07': 34,
 '11': 58}

In [6]:
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the Average Number of Comments for Ask HN Posts by Hour
This will help us identify the best time to create a post to draw a maximum of comments

In [7]:
avg_by_hour = []

for key in comments_by_hour:
    avg_comments_by_hour = comments_by_hour[key]/ counts_by_hour[key]
    avg_by_hour.append([key, round(avg_comments_by_hour, 3)])
    
avg_by_hour

[['09', 5.578],
 ['13', 14.741],
 ['10', 13.441],
 ['14', 13.234],
 ['16', 16.796],
 ['23', 7.985],
 ['12', 9.411],
 ['17', 11.46],
 ['15', 38.595],
 ['21', 16.009],
 ['20', 21.525],
 ['02', 23.81],
 ['18', 13.202],
 ['03', 7.796],
 ['05', 10.087],
 ['19', 10.8],
 ['01', 11.383],
 ['22', 6.746],
 ['08', 10.25],
 ['04', 7.17],
 ['00', 8.127],
 ['06', 9.023],
 ['07', 7.853],
 ['11', 11.052]]

## Sorting and Printing Values from a List of Lists

In [8]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
swap_avg_by_hour

[[5.578, '09'],
 [14.741, '13'],
 [13.441, '10'],
 [13.234, '14'],
 [16.796, '16'],
 [7.985, '23'],
 [9.411, '12'],
 [11.46, '17'],
 [38.595, '15'],
 [16.009, '21'],
 [21.525, '20'],
 [23.81, '02'],
 [13.202, '18'],
 [7.796, '03'],
 [10.087, '05'],
 [10.8, '19'],
 [11.383, '01'],
 [6.746, '22'],
 [10.25, '08'],
 [7.17, '04'],
 [8.127, '00'],
 [9.023, '06'],
 [7.853, '07'],
 [11.052, '11']]

In [9]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
sorted_swap

[[38.595, '15'],
 [23.81, '02'],
 [21.525, '20'],
 [16.796, '16'],
 [16.009, '21'],
 [14.741, '13'],
 [13.441, '10'],
 [13.234, '14'],
 [13.202, '18'],
 [11.46, '17'],
 [11.383, '01'],
 [11.052, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.087, '05'],
 [9.411, '12'],
 [9.023, '06'],
 [8.127, '00'],
 [7.985, '23'],
 [7.853, '07'],
 [7.796, '03'],
 [7.17, '04'],
 [6.746, '22'],
 [5.578, '09']]

In [10]:
print("Top 5 Hours for `Ask Posts` Comments:\n")

for avg, hr in sorted_swap[:5]:
    time = dt.datetime.strptime(hr, "%H").strftime('%H:%M')
    print('{hour}: {comments:.2f} average comments per post'.format(hour = time, comments = avg))

Top 5 Hours for `Ask Posts` Comments:

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. This is up to 2.5 times the average number of comments for `Ask HN` posts. Also, there's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Conclusion
In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.